In [1]:
using Pkg
Pkg.activate(".")
Pkg.instantiate()

  Activating project at `c:\Dev\amo-personal-project`


In [2]:
# Pkg.add("JuMP")
# Pkg.add("HiGHS")
# Pkg.add("Plots")
using JuMP, HiGHS, Plots

In [3]:
# Define all sets
T = collect(1:2) # time periods
n_N = 3 # number of nodes
N = collect(1:n_N) # nodes
G = collect(1:3) # generators
L = collect(1:2) # transmission lines

2-element Vector{Int64}:
 1
 2

In [4]:
# Define connections between nodes as an upper triangular matrix
orig_n = [1,1]
dest_n = [2,3]

lines = zeros(n_N, n_N)

for i in 1:size(orig_n)[1]
    lines[orig_n[i], dest_n[i]] = i # incoming transmission lines
    lines[dest_n[i], orig_n[i]] = -i # outgoing transmission lines
end
lines

3×3 Matrix{Float64}:
  0.0  1.0  2.0
 -1.0  0.0  0.0
 -2.0  0.0  0.0

In [5]:
function get_lines_for_node(node, incoming)
    if incoming
        return filter(i -> i >= 1.0, lines[node, 1:end])
    end
    return filter(i -> i <= -1.0, lines[node, 1:end])
end

get_lines_for_node (generic function with 1 method)

In [6]:
# Parameters
c = [10, 1, 1]
P_max = [ # maximum generation capacity of each generator
    [200, 200], # convetional generators
    [150, 50], # renewable generators
    [150, 0]
] 
F_max = [200, 200] # transmission line capacity
B_max = [50, 50, 50] # battery capacity
D = [150, 100, 0] # demand 

3-element Vector{Int64}:
 150
 100
   0

In [7]:
# Location of the generators
loc_G = [1,2,3]

3-element Vector{Int64}:
 1
 2
 3

In [8]:
function get_generator_for_node(node)
    return findall(i -> i == node, loc_G)
end

get_generator_for_node (generic function with 1 method)

In [9]:
vcat([0],T)

3-element Vector{Int64}:
 0
 1
 2

In [10]:
model = Model(HiGHS.Optimizer)

@variable(model, P_max[g][t] >= P[g in G, t in T] >= 0) 
@variable(model, F_max[l] >= F[l in L, t in T] >= -F_max[l])

@variable(model, B_max[n] >= B[n in N, t in vcat([0],T)] >= 0) 
@variable(model, ΔB[n in N, t in T])

@constraint(model, init_battery_charge[n in N], B[n, 0] == 0)
@constraint(model, battery_charge[n in N, t in T], B[n,t-1] + ΔB[n,t] == B[n, t])

@constraint(model, power_flow[n in N, t in T], 
    sum(P[g,t] for g in get_generator_for_node(n))
    - D[n]
    + sum(F[i,t] for i in get_lines_for_node(n, true))
    - sum(F[-i,t] for i in get_lines_for_node(n, false))
    - ΔB[n,t]
    == 0
)

@objective(model, Min, sum(c[g]*P[g,t] for g in G, t in T))

print(model)

Min 10 P[1,1] + 10 P[1,2] + P[2,1] + P[2,2] + P[3,1] + P[3,2]
Subject to


 init_battery_charge[1] : B[1,0] == 0.0
 init_battery_charge[2] : B[2,0] == 0.0
 init_battery_charge[3] : B[3,0] == 0.0
 battery_charge[1,1] : B[1,0] - B[1,1] + ΔB[1,1] == 0.0
 battery_charge[2,1] : B[2,0] - B[2,1] + ΔB[2,1] == 0.0
 battery_charge[3,1] : B[3,0] - B[3,1] + ΔB[3,1] == 0.0
 battery_charge[1,2] : B[1,1] - B[1,2] + ΔB[1,2] == 0.0
 battery_charge[2,2] : B[2,1] - B[2,2] + ΔB[2,2] == 0.0
 battery_charge[3,2] : B[3,1] - B[3,2] + ΔB[3,2] == 0.0
 power_flow[1,1] : P[1,1] + F[1,1] + F[2,1] - ΔB[1,1] == 150.0
 power_flow[2,1] : P[2,1] - F[1,1] - ΔB[2,1] == 100.0
 power_flow[3,1] : P[3,1] - F[2,1] - ΔB[3,1] == 0.0
 power_flow[1,2] : P[1,2] + F[1,2] + F[2,2] - ΔB[1,2] == 150.0
 power_flow[2,2] : P[2,2] - F[1,2] - ΔB[2,2] == 100.0
 power_flow[3,2] : P[3,2] - F[2,2] - ΔB[3,2] == 0.0
 P[1,1] >= 0.0
 P[2,1] >= 0.0
 P[3,1] >= 0.0
 P[1,2] >= 0.0
 P[2,2] >= 0.0
 P[3,2] >= 0.0
 F[1,1] >= -200.0
 F[2,1] >= -200.0
 F[1,2] >= -200.0
 F[2,2] >= -200.0
 B[1,0] >= 0.0
 B[2,0] >= 0.0
 B[3,0] >= 0.0

In [11]:
optimize!(model)

Running HiGHS 1.4.2 [date: 1970-01-01, git hash: f797c1ab6]
Copyright (c) 2022 ERGO-Code under MIT licence terms
Presolving model
9 rows, 18 cols, 28 nonzeros
5 rows, 14 cols, 20 nonzeros
5 rows, 13 cols, 19 nonzeros
Presolve : Reductions: rows 5(-10); columns 13(-12); elements 19(-22)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -2.1921323676e-05 Pr: 3(600); Du: 0(2.87425e-13) 0s
          6     1.8500000000e+03 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 6
Objective value     :  1.8500000000e+03
HiGHS run time      :          0.01


In [12]:
print(value.(P))
print(value.(B))
print(value.(ΔB))
print(value.(F))

2-dimensional DenseAxisArray{Float64,2,...} with index sets:
    Dimension 1, [1, 2, 3]
    Dimension 2, [1, 2]
And data, a 3×2 Matrix{Float64}:
   0.0  150.0
 150.0   50.0
 150.0    0.02-dimensional DenseAxisArray{Float64,2,...} with index sets:
    Dimension 1, [1, 2, 3]
    Dimension 2, [0, 1, 2]
And data, a 3×3 Matrix{Float64}:
 -0.0  50.0  0.0
 -0.0  -0.0  0.0
 -0.0   0.0  0.02-dimensional DenseAxisArray{Float64,2,...} with index sets:
    Dimension 1, [1, 2, 3]
    Dimension 2, [1, 2]
And data, a 3×2 Matrix{Float64}:
 50.0  -50.0
 -0.0   -0.0
 -0.0   -0.02-dimensional DenseAxisArray{Float64,2,...} with index sets:
    Dimension 1, [1, 2]
    Dimension 2, [1, 2]
And data, a 2×2 Matrix{Float64}:
  50.0  -50.0
 150.0   -0.0